# Main Visualization

## Exploration

In [1]:
# Load in dataset
import pandas as pd
import altair as alt
df = pd.read_csv("https://raw.githubusercontent.com/wadefagen/datasets/master/gpa/uiuc-gpa-dataset.csv")

In [2]:
df['Students'] = df['A+'] + df['A'] + df['A-'] + df['B+'] + df['B'] + df['B-'] + df['C+'] + df['C'] + df['C-'] + df['D+'] + df['D'] + df['D-'] + df['F']

In [3]:
# Then, use the students column to find Average GPA 
# (scale: https://registrar.illinois.edu/courses-grades/explanation-of-grades/)
df['Average GPA'] = (df['A+'] * 4.0 + df['A'] * 4.0 + df['A-'] * 3.67 \
                     + df['B+'] * 3.33 + df['B'] * 3.0 + df['B-'] * 2.67 \
                     + df['C+'] * 2.33 + df['C'] * 2.0 + df['C-'] * 1.67 \
                     + df['D+'] * 1.33 + df['D'] * 1.0 + df['D-'] * 0.67) / df['Students']

In [4]:
df = df[df['Sched Type'] == 'LEC']

In [5]:
# Group the data by course title and count the number of unique primary instructors
course_counts = df.groupby('Course Title')['Primary Instructor'].nunique()

# Filter the dataframe to include only courses with multiple instructors
df = df[df['Course Title'].isin(course_counts[course_counts > 1].index)]
df

,Year,Term,YearTerm,Subject,Number,Course Title,Sched Type,A+,A,A-,...,C,C-,D+,D,D-,F,W,Primary Instructor,Students,Average GPA
5,2023,Spring,2023-sp,ABE,141,ABE Principles: Biological,LEC,0,5,4,...,3,1,0,1,0,0,0,"Leonelli, Laurie B",27,3.098889
106,2023,Spring,2023-sp,ACE,210,Environmental Economics,LEC,10,19,13,...,9,5,1,2,1,0,0,"Damon, Lisa A",89,3.101573
113,2023,Spring,2023-sp,ACE,262,App Stat Mthds & Data AnlytcsI,LEC,13,13,8,...,7,4,2,2,1,0,0,"Gundersen, Lisa C",71,3.122394
115,2023,Spring,2023-sp,ACE,270,Consumer Economics,LEC,20,16,8,...,0,1,0,0,0,0,0,"Gundersen, Lisa C",58,3.712931
116,2023,Spring,2023-sp,ACE,300,Interm. Applied Microeconomics,LEC,12,20,10,...,10,4,4,4,1,1,0,"Endres, Renata",98,3.003571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69063,2010,Summer,2010-su,STAT,400,Statistics and Probability I,LEC,4,15,7,...,2,2,0,1,0,3,1,"Monrad, Ditlev",47,3.142553
69064,2010,Summer,2010-su,STAT,410,Statistics and Probability II,LEC,5,10,2,...,1,3,0,0,0,2,1,"Stepanov, Alexei G",31,3.183226
69065,2010,Summer,2010-su,STAT,440,Statistical Data Management,LEC,4,12,8,...,0,0,0,0,0,0,0,"Unger, David",28,3.774643
69066,2010,Summer,2010-su,TAM,212,Introductory Dynamics,LEC,0,1,3,...,1,1,0,2,0,1,0,"Morgan, William T",28,2.595714


In [6]:
popular_course = df[(df.Subject == 'CS') & (df.Number == 225)] 

In [7]:
# Add a row number within each group of 'Primary Instructor' and 'YearTerm' for uniqueness
popular_course['RowNumber'] = popular_course.groupby(['Primary Instructor', 'YearTerm']).cumcount() + 1

# Create a detailed section identifier
popular_course['DetailedSection'] = (
    popular_course['Primary Instructor'] + ' - ' +
    popular_course['YearTerm'] + ' - ' +
    popular_course['Number'].astype(str) + ' - ' +
    popular_course['RowNumber'].astype(str)
)

chart = alt.Chart(popular_course).mark_bar().encode(
    x=alt.X('DetailedSection:N', axis=alt.Axis(title='Section')),
    y=alt.Y('Average GPA:Q', axis=alt.Axis(title='Average GPA')),
    color=alt.Color('Primary Instructor:N', legend=alt.Legend(title="Instructor")),
    tooltip=['DetailedSection', 'Average GPA']
).properties(
    width=800,
    height=400,
    title='Average GPA for Each Section of CS 225 Taught by Instructor'
)

chart

C:\Users\echen\AppData\Local\Temp\ipykernel_32132\2512238875.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  popular_course['RowNumber'] = popular_course.groupby(['Primary Instructor', 'YearTerm']).cumcount() + 1
C:\Users\echen\AppData\Local\Temp\ipykernel_32132\2512238875.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  popular_course['DetailedSection'] = (


alt.Chart(...)

## Creating Final Version w/ Interactivity to Select Course

In [8]:
# To support interactivity, make CS courses
CS_Courses = df[(df.Subject == 'CS') & (df.Number.isin([105, 173, 225, 374, 446]))] 

# Create dropdown and selection for course number
dropdown = alt.binding_select(options=[105, 173, 225, 374, 446], 
                              labels=["CS 105", "CS 173", "CS 225", "CS 374", "CS 446"], 
                              name="Course: ",) 

course_selection = alt.selection_point(fields=['Number'], 
                                       bind=dropdown, 
                                       name="Course: ", 
                                       value=225) 

# Add a row number within each group of 'Primary Instructor' and 'YearTerm' for uniqueness
CS_Courses['RowNumber'] = CS_Courses.groupby(['Primary Instructor', 'YearTerm']).cumcount() + 1

# Create a detailed section identifier
CS_Courses['DetailedSection'] = (
    CS_Courses['Primary Instructor'] + ' - ' +
    CS_Courses['YearTerm'] + ' - ' +
    CS_Courses['Number'].astype(str) + ' - ' +
    CS_Courses['RowNumber'].astype(str)
)

chart = alt.Chart(CS_Courses).mark_bar().encode(
    x=alt.X('DetailedSection:N', axis=alt.Axis(title='Section')),
    y=alt.Y('Average GPA:Q', axis=alt.Axis(title='Average GPA')),
    color=alt.Color('Primary Instructor:N', legend=alt.Legend(title="Instructor")),
    tooltip=['DetailedSection', 'Average GPA']
).properties(
    width=800,
    height=400,
    title=f'Average GPA for Each Class Section Taught by Instructor'
).add_params(
    course_selection
).transform_filter(
    course_selection
).configure_title(
    fontSize=16
)

chart
chart.save('final-proj-viz1.json', format='json')

C:\Users\echen\AppData\Local\Temp\ipykernel_32132\827556400.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CS_Courses['RowNumber'] = CS_Courses.groupby(['Primary Instructor', 'YearTerm']).cumcount() + 1
C:\Users\echen\AppData\Local\Temp\ipykernel_32132\827556400.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CS_Courses['DetailedSection'] = (


For the writeup, I was going to introduce something along the lines of "Does the Professor teaching really matter?", in this visualization we're presenting a case study that showcases a class that many students have taken that has had many different instructors. You can easily examine the chart by interacting with it to see the disparities.

In [9]:
df.Subject.nunique()

95

In [10]:
df

,Year,Term,YearTerm,Subject,Number,Course Title,Sched Type,A+,A,A-,...,C,C-,D+,D,D-,F,W,Primary Instructor,Students,Average GPA
5,2023,Spring,2023-sp,ABE,141,ABE Principles: Biological,LEC,0,5,4,...,3,1,0,1,0,0,0,"Leonelli, Laurie B",27,3.098889
106,2023,Spring,2023-sp,ACE,210,Environmental Economics,LEC,10,19,13,...,9,5,1,2,1,0,0,"Damon, Lisa A",89,3.101573
113,2023,Spring,2023-sp,ACE,262,App Stat Mthds & Data AnlytcsI,LEC,13,13,8,...,7,4,2,2,1,0,0,"Gundersen, Lisa C",71,3.122394
115,2023,Spring,2023-sp,ACE,270,Consumer Economics,LEC,20,16,8,...,0,1,0,0,0,0,0,"Gundersen, Lisa C",58,3.712931
116,2023,Spring,2023-sp,ACE,300,Interm. Applied Microeconomics,LEC,12,20,10,...,10,4,4,4,1,1,0,"Endres, Renata",98,3.003571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69063,2010,Summer,2010-su,STAT,400,Statistics and Probability I,LEC,4,15,7,...,2,2,0,1,0,3,1,"Monrad, Ditlev",47,3.142553
69064,2010,Summer,2010-su,STAT,410,Statistics and Probability II,LEC,5,10,2,...,1,3,0,0,0,2,1,"Stepanov, Alexei G",31,3.183226
69065,2010,Summer,2010-su,STAT,440,Statistical Data Management,LEC,4,12,8,...,0,0,0,0,0,0,0,"Unger, David",28,3.774643
69066,2010,Summer,2010-su,TAM,212,Introductory Dynamics,LEC,0,1,3,...,1,1,0,2,0,1,0,"Morgan, William T",28,2.595714


# Contextual Visualizations
Using the Graybook Prof. Appointment Data from CS

In [11]:
graybook = pd.read_csv('https://raw.githubusercontent.com/zuyouchen/107_datasets/main/graybook/graybook-clean-sp23.csv')
graybook

,Employee Name,Job Title,Tenure,Empl Class,Present FTE,Proposed FTE,Present Salary,Proposed Salary,College Code,College Name,Department Code,Department Name,Job Titles,Employee Classes
0,"Gerstenecker, David G",DIR OF INFO TECHNOL,NaN,BA,1.0,1.0,106476.00,109670.28,KL,"Agr, Consumer & Env Sciences",348,ACES Technology Services,['DIR OF INFO TECHNOL'],['BA']
1,"Henry, Joshua Keith",ASSOC DIR INFO TECHNOL,NaN,BA,1.0,1.0,80871.33,83701.83,KL,"Agr, Consumer & Env Sciences",348,ACES Technology Services,['ASSOC DIR INFO TECHNOL'],['BA']
2,"Meyer, Donald L",SR NETWORK ANLYS,NaN,BA,1.0,1.0,89558.22,92692.76,KL,"Agr, Consumer & Env Sciences",348,ACES Technology Services,['SR NETWORK ANLYS'],['BA']
3,"Schwartz, Sarah",COMM COORD,NaN,BA,1.0,1.0,50000.00,51750.00,KL,"Agr, Consumer & Env Sciences",521,ADM Institute for PHL,['COMM COORD'],['BA']
4,"Winter-Nelson, Alex E","DIR, ADMI",NaN,NaN,1.0,1.0,186666.00,193532.64,KL,"Agr, Consumer & Env Sciences",521,ADM Institute for PHL,"['DIR, ADMI', 'ASSOC DEAN FOR INATL PRGMS', 'P...","['AA', 'BA']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6014,"Chambers, Sonya L",EXEC DIR STRATG INITIATIVES,NaN,BA,1.0,1.0,163200.00,169500.00,NV,Vice Chancellor for Diversity,594,Vice Chancellor for Diversity,['EXEC DIR STRATG INITIATIVES'],['BA']
6015,"Garrick, Sean C","VC FOR DVSTY, EQUITY & INCL",NaN,NaN,1.0,1.0,338248.98,351778.94,NV,Vice Chancellor for Diversity,594,Vice Chancellor for Diversity,"['VC FOR DVSTY, EQUITY & INCL']",['BA']
6016,"Guerra Perez, Gioconda",EXEC ASSOC VC FOR DVSTY,NaN,BA,1.0,1.0,178500.00,187250.00,NV,Vice Chancellor for Diversity,594,Vice Chancellor for Diversity,['EXEC ASSOC VC FOR DVSTY'],['BA']
6017,"VanDuyne, Adam Jareb",EXEC DIR FIN & ADMN,NaN,BA,1.0,1.0,141000.00,147200.00,NV,Vice Chancellor for Diversity,594,Vice Chancellor for Diversity,['EXEC DIR FIN & ADMN'],['BA']


In [12]:
# CS is department code 434
graybook_CS = graybook[graybook['Department Code'] == 434]
graybook_CS

,Employee Name,Job Title,Tenure,Empl Class,Present FTE,Proposed FTE,Present Salary,Proposed Salary,College Code,College Name,Department Code,Department Name,Job Titles,Employee Classes
2573,"Abdelzaher, Tarek",PROF,NaN,NaN,1.0,1.0,208154.0,217521.0,KP,Grainger Engineering,434,Computer Science,"['PROF', 'PROF, ITI', 'PROF, CSL']",['AA']
2574,"Adve, Sarita V",PROF,NaN,NaN,1.0,1.0,218504.0,230000.0,KP,Grainger Engineering,434,Computer Science,"['PROF', 'PROF, ITI', 'PROF, CSL']",['AA']
2575,"Adve, Vikram Sadanand",PROF,NaN,NaN,1.0,1.0,218462.0,230000.0,KP,Grainger Engineering,434,Computer Science,"['PROF, NCSA', 'PROF', 'PROF, ITI', 'PROF, CSL']",['AA']
2576,"Alawini, Abdussalam Alhadi M",TCH ASST PROF,M,AA,1.0,1.0,105575.0,110326.0,KP,Grainger Engineering,434,Computer Science,['TCH ASST PROF'],['AA']
2577,"Amato, Nancy Marie",HEAD,NaN,NaN,1.0,1.0,272500.0,289238.0,KP,Grainger Engineering,434,Computer Science,"['HEAD', 'PROF', 'PROF, HCESC', 'PROF, NCSA', ...","['AA', 'BA']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2725,"Zhai, Chengxiang",PROF,NaN,NaN,1.0,1.0,187107.0,195527.0,KP,Grainger Engineering,434,Computer Science,"['PROF, IGB', 'WILLETT FACULTY SCHOLAR', 'PROF']",['AA']
2726,"Zhang, Lingming",ASSOC PROF,A,AA,1.0,1.0,140600.0,146927.0,KP,Grainger Engineering,434,Computer Science,['ASSOC PROF'],['AA']
2727,"Zhao, Han",ASST PROF,NaN,NaN,1.0,1.0,125000.0,130625.0,KP,Grainger Engineering,434,Computer Science,['ASST PROF'],['AA']
2728,"Zike, Heather",UGRAD PRGM COORD & ACAD ADVR,NaN,BA,1.0,1.0,62968.4,65487.4,KP,Grainger Engineering,434,Computer Science,['UGRAD PRGM COORD & ACAD ADVR'],['BA']


In [13]:
# Narrow down to only include instructors who have 'PROF' in their Job Title
def profInJobTitle(jobTitle):
    if 'PROF' in jobTitle:
        return True
    return False

# Apply the function to the Job Titles column
graybook_CS.loc[:, 'Teaching'] = graybook_CS['Job Title'].apply(profInJobTitle)
# Subset the dataframe to only include instructors who have 'PROF' in their Job Title
graybook_CS_prof = graybook_CS[graybook_CS['Teaching'] == True]
graybook_CS_prof 

C:\Users\echen\AppData\Local\Temp\ipykernel_32132\1781993938.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  graybook_CS.loc[:, 'Teaching'] = graybook_CS['Job Title'].apply(profInJobTitle)


,Employee Name,Job Title,Tenure,Empl Class,Present FTE,Proposed FTE,Present Salary,Proposed Salary,College Code,College Name,Department Code,Department Name,Job Titles,Employee Classes,Teaching
2573,"Abdelzaher, Tarek",PROF,NaN,NaN,1.0,1.0,208154.0,217521.0,KP,Grainger Engineering,434,Computer Science,"['PROF', 'PROF, ITI', 'PROF, CSL']",['AA'],True
2574,"Adve, Sarita V",PROF,NaN,NaN,1.0,1.0,218504.0,230000.0,KP,Grainger Engineering,434,Computer Science,"['PROF', 'PROF, ITI', 'PROF, CSL']",['AA'],True
2575,"Adve, Vikram Sadanand",PROF,NaN,NaN,1.0,1.0,218462.0,230000.0,KP,Grainger Engineering,434,Computer Science,"['PROF, NCSA', 'PROF', 'PROF, ITI', 'PROF, CSL']",['AA'],True
2576,"Alawini, Abdussalam Alhadi M",TCH ASST PROF,M,AA,1.0,1.0,105575.0,110326.0,KP,Grainger Engineering,434,Computer Science,['TCH ASST PROF'],['AA'],True
2578,"Angrave, Lawrence Christopher",TCH PROF,M,AA,1.0,1.0,147088.0,153707.0,KP,Grainger Engineering,434,Computer Science,['TCH PROF'],['AA'],True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2723,"Yuile, Adam Bates",ASSOC PROF,NaN,NaN,1.0,1.0,129540.0,143369.0,KP,Grainger Engineering,434,Computer Science,"['ASSOC PROF, ITI', 'ASSOC PROF']",['AA'],True
2725,"Zhai, Chengxiang",PROF,NaN,NaN,1.0,1.0,187107.0,195527.0,KP,Grainger Engineering,434,Computer Science,"['PROF, IGB', 'WILLETT FACULTY SCHOLAR', 'PROF']",['AA'],True
2726,"Zhang, Lingming",ASSOC PROF,A,AA,1.0,1.0,140600.0,146927.0,KP,Grainger Engineering,434,Computer Science,['ASSOC PROF'],['AA'],True
2727,"Zhao, Han",ASST PROF,NaN,NaN,1.0,1.0,125000.0,130625.0,KP,Grainger Engineering,434,Computer Science,['ASST PROF'],['AA'],True


In [14]:
# Reduce the number of columns to only be when the professor has a non-unique 'Job Title' 
graybook_CS_prof_nonunique = graybook_CS_prof[graybook_CS_prof['Job Title'].duplicated(keep=False)]

# Replace job title acronyms to avoid jargon
title_mapping = {"ASST": "Assistant", 
                 "PROF": "Professor", 
                 "RES": "Research", 
                 "TCH": "Teaching", 
                 "ASSOC": "Associate"}

graybook_CS_prof_nonunique.loc[:,'Job Title'] = graybook_CS_prof_nonunique['Job Title'].replace(title_mapping, regex=True)

# Chart that shows a count of the professors in job titles, ordered by count

chart2 = alt.Chart(graybook_CS_prof_nonunique).mark_bar().encode(
    x=alt.X('Job Title:N', axis=alt.Axis(title='Job Title', labelAngle=20, labelOffset=-15, titlePadding=10), sort='-y'),
    y=alt.Y('count():Q', axis=alt.Axis(title='Number of Professors', titlePadding=10)),
    color=alt.Color('Job Title:N', legend=alt.Legend(title="Job Title")),
    tooltip=[alt.Tooltip('Job Title:N', title='Job Title'), alt.Tooltip('count():Q', title='Number of Professors')]
).properties(
    width=800,
    height=400,
    title='Job Titles of Spring 2023 CS Professors',
).configure_title(
    fontSize=16  # Adjust fontSize as needed
)

chart2.save('final-proj-3.1-contextual-job-titles.json', format='json')

In [15]:
graybook_CS_prof['Tenure'].unique()

array([nan, 'M', 'A', 'P'], dtype=object)

In [16]:
# According to https://www.trustees.uillinois.edu/trustees/resources/historical-files/GrayBook2022.pdf,
''' 
Tenure Status Meanings
M Multi-Year Contract Agreement
A Indefinite Tenure
P Probationary Term
'''
# Replace NaN values with "unknown"
graybook_CS_prof.loc[:, 'Tenure'] = graybook_CS_prof['Tenure'].fillna('Unknown')
# Replace M, A, P with their full meanings
tenure_mapping = {"M": "Multi-Year Contract Agreement", 
                 "A": "Indefinite Tenure", 
                 "P": "Probationary Term"}
graybook_CS_prof.loc[:,'Tenure'] = graybook_CS_prof['Tenure'].replace(tenure_mapping, regex=True)
graybook_CS_prof

,Employee Name,Job Title,Tenure,Empl Class,Present FTE,Proposed FTE,Present Salary,Proposed Salary,College Code,College Name,Department Code,Department Name,Job Titles,Employee Classes,Teaching
2573,"Abdelzaher, Tarek",PROF,Unknown,NaN,1.0,1.0,208154.0,217521.0,KP,Grainger Engineering,434,Computer Science,"['PROF', 'PROF, ITI', 'PROF, CSL']",['AA'],True
2574,"Adve, Sarita V",PROF,Unknown,NaN,1.0,1.0,218504.0,230000.0,KP,Grainger Engineering,434,Computer Science,"['PROF', 'PROF, ITI', 'PROF, CSL']",['AA'],True
2575,"Adve, Vikram Sadanand",PROF,Unknown,NaN,1.0,1.0,218462.0,230000.0,KP,Grainger Engineering,434,Computer Science,"['PROF, NCSA', 'PROF', 'PROF, ITI', 'PROF, CSL']",['AA'],True
2576,"Alawini, Abdussalam Alhadi M",TCH ASST PROF,Multi-Year Contract Agreement,AA,1.0,1.0,105575.0,110326.0,KP,Grainger Engineering,434,Computer Science,['TCH ASST PROF'],['AA'],True
2578,"Angrave, Lawrence Christopher",TCH PROF,Multi-Year Contract Agreement,AA,1.0,1.0,147088.0,153707.0,KP,Grainger Engineering,434,Computer Science,['TCH PROF'],['AA'],True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2723,"Yuile, Adam Bates",ASSOC PROF,Unknown,NaN,1.0,1.0,129540.0,143369.0,KP,Grainger Engineering,434,Computer Science,"['ASSOC PROF, ITI', 'ASSOC PROF']",['AA'],True
2725,"Zhai, Chengxiang",PROF,Unknown,NaN,1.0,1.0,187107.0,195527.0,KP,Grainger Engineering,434,Computer Science,"['PROF, IGB', 'WILLETT FACULTY SCHOLAR', 'PROF']",['AA'],True
2726,"Zhang, Lingming",ASSOC PROF,Indefinite Tenure,AA,1.0,1.0,140600.0,146927.0,KP,Grainger Engineering,434,Computer Science,['ASSOC PROF'],['AA'],True
2727,"Zhao, Han",ASST PROF,Unknown,NaN,1.0,1.0,125000.0,130625.0,KP,Grainger Engineering,434,Computer Science,['ASST PROF'],['AA'],True


In [17]:
# Now, generating visualization showing distribution of tenure status of CS professors
chart3 = alt.Chart(graybook_CS_prof).mark_bar().encode(
    x=alt.X('Tenure:N', axis=alt.Axis(title='Tenure Status', labelAngle=0, titlePadding=10)),
    y=alt.Y('count():Q', axis=alt.Axis(title='Number of Professors', titlePadding=10)),
    color=alt.Color('Tenure:N', legend=alt.Legend(title="Tenure Status")),
    tooltip=[alt.Tooltip('Tenure:N', title='Tenure Status'), alt.Tooltip('count():Q', title='Number of Professors')]
).properties(
    width=800,
    height=400,
    title='Tenure Status of Spring 2023 CS Professors'
).configure_title(
    fontSize=16
)

chart3.save('final-proj-3.1-contextual-tenure.json', format='json')